In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', "r").read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte,  Yte  = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
n_emb = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g)
W1 = torch.randn((n_emb * block_size, n_hidden), generator=g)* (5/3)/((n_emb*block_size)**0.5)
#b1 = torch.randn(n_hidden, generator=g)*0.01
W2 = torch.randn((n_hidden, vocab_size), generator=g)*0.01
b2 = torch.randn(vocab_size, generator=g)*0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

12097


In [8]:
max_steps = 100000
batch_size = 32
lossi = []

for i in range(max_steps):
    
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 # + b1
    
    hpreact = bngain*(hpreact - hpreact.mean(0, keepdim=True))/hpreact.std(0, keepdim=True) + bnbias
    
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)
    
    for p in parameters:
        p.grad = None
    loss.backward()
    
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 100000: 3.3239
  10000/ 100000: 2.0322
  20000/ 100000: 2.5675
  30000/ 100000: 2.0125
  40000/ 100000: 2.2446
  50000/ 100000: 1.8897
  60000/ 100000: 2.0785
  70000/ 100000: 2.3681
  80000/ 100000: 2.2918
  90000/ 100000: 2.0238


In [9]:
plt.plot(lossi)

In [10]:
@torch.no_grad
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngain*(hpreact - hpreact.mean(0, keepdim=True))/hpreact.std(0, keepdim=True) + bnbias
    h = torch.tanh(hpreact)   
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
split_loss('train')
split_loss('val')

NameError: name 'b1' is not defined

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for _ in range(1000):
    
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        p = F.softmax(logits, dim=1)
        ix = torch.multinomial(p, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
        
    print(''.join(itos[i] for i in out))

trezlakshlvirkylahntyh.
brinncjtna.
trlusskzkanna.
strighubrthviyvonts.
mbrisslnvukslatrdygvannysmessdelyn.
bvgrry.
fsbstthllivendtallaslumslfx.
steslegrisaivannikazdyibellamrierriffrithk.
chrishrabrayrh.
khmbrqevenslexx.
mahrishnvomarlislmaruljuschniya.
shmbitjapxrengshlsceffernnysilmarlhkirrissahmarlikamrismanlissanfarillanghtfiyornnverrezshvenslevannalmilliyatnsissshdugnnishbuwfrahsharlakshrudnstorgkzisha.
rddnty.
tre.
shfard.
tra.
jmakdhlsh.
frrighthrlrklmigrnghnnnysaydann.
riffossln.
merrntll.
dthrishvandahmyghn.
roserztvijriffrosc.
krrjkasmbaavidnsomidhliegrgbraylinfslifefrrowtsadrikaswrrosshlsssttt.
briyannasullwaturzcqnajmisslavcarlieghz.
lhnnyquogulkdrn.
jsmarrtygrnsshrisptanlexikelss.
frrisraagatxicfriqifgsdvellanny.
suhmenf.
rikelddndalifw.
tarressmanolulszadylester.
tlscaznyxmeghtly.
miconnisomszfightlsxjsannyxannatrijsoknnyssndrok.
ryardhetzakszaclifr.
chndsislavaksylandress.
fern.
driffrsprayna.
thavnzelizandusslingstzy.
tthi.
dumritza.
quiszaciss.
jequarllisusbbder.
mrti